In [2]:
import sys
sys.path.append('../scripts/')

from time import time
import glob
import warnings
import param

import numpy as np
import pandas as pd
import xarray as xr
from scipy.special import gammaln
from scipy.optimize import curve_fit, minimize
from scipy.stats import pearsonr

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.xarray

from epsilon_tools import *
from tools import str2date, avg_funs

import warnings
warnings.simplefilter("ignore",category=FutureWarning)
warnings.simplefilter("ignore",category=RuntimeWarning)

# set up figure params
sns.set(style='ticks', context='notebook', palette='colorblind',font_scale=1.5)
mpl.rc('figure', dpi=100, figsize=[6, 6])
mpl.rc('savefig', dpi=500, bbox='tight')
mpl.rc('legend', frameon=False)

%matplotlib inline

In [3]:
dsv = xr.open_dataset('../data/chi/variable_dof.nc')
dsf = xr.open_dataset('../data/chi/fixed_dof.nc')

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/sebastianessink/Dropbox (MIT)/epsilon/data/chi/variable_dof.nc'

In [ ]:
dsv.isel(time=0).dof.plot(marker='o')

In [ ]:
def plot_hist(ax, ds):
    ind, bins = np.histogram(ds.eps1_kra, bins=30, range=(1e-12,1e-7));
    logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
    ds.eps1_kra.plot.hist(xscale='log', bins=logbins, ax=ax);

In [ ]:
f,ax = plt.subplots(1,2, figsize=(12,5))
plot_hist(ax[0],dsv)
plot_hist(ax[1],dsf)

In [ ]:
ds = xr.Dataset()
ds['variable'] = dsv.eps1_kra
ds['fixed'] = dsf.eps1_kra
ds['rc'] = dsf.eps_rc

In [ ]:
ind, bins = np.histogram(ds.fixed, bins=30, range=(1e-12,1e-7));
logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))

g = sns.jointplot(x='fixed',y='variable',data=ds, kind='scatter',dropna=True,
             xlim=(1e-13,1e-8),ylim=(1e-13,1e-8), marginal_kws={'bins':logbins} )
g.ax_joint.set_xscale('log')
g.ax_joint.set_yscale('log')
g.ax_joint.set_xlabel('Fixed DoF')
g.ax_joint.set_ylabel('Variable DoF')

g.ax_joint.plot(ds.fixed, ds.fixed, color='k', lw=0.2)

plt.savefig('../figures/fixed_vs_variable_dof.pdf')

In [ ]:
mask = (dsf.eps1_bat < 3.05e-9) & (dsf.eps1_bat > 3.04e-9)

In [ ]:
# ds = ds dsf.where(mask)
ds = dsv

ds.dropna(dim='time', how='all')
ds['itime'] = ('time',np.arange(0,len(ds.time)))

hover_cols = ['eps1_bat','eps1_kra','eps2_bat','eps2_kra', 'chi1']
arguments = {'hover_cols':hover_cols,'legend':'left','groupby':'itime','loglog':True}

obs = ds.hvplot(x='k_rpm',y='corrdTdzsp1_rpm',label='Observed',**arguments)
noi = ds.hvplot(x='k_rpm',y='noise_rpm',label='Noise',line_dash='dotted',**arguments)
bat = ds.hvplot(x='k_rpm',y='bat1',label='Batchelor',**arguments)
krai = ds.hvplot(x='k_rpm',y='kra1',label='Kraichnan',**arguments)
power = ds.hvplot(x='k_rpm',y='power1',label='Power law',line_dash='dashed',**arguments)
rc = ds.hvplot(x='k_rpm',y='bat1_rc',label='Ren-Chieh',**arguments)

In [ ]:
a = (obs*bat*krai*rc*power*noi).options(height=500,width=400,ylim=(1e-9, 1e-2),xlim=(1e1,2e3),
                      title=f'Variable DoF',
                      xlabel=r'k_z',ylabel='dPhidz',
                      fontsize={'title': 16, 'labels': 14, 'xticks': 12, 'yticks': 12, 'legend':12})

In [ ]:
# ds = ds dsf.where(mask)
ds = dsf

ds.dropna(dim='time', how='all')
ds['itime'] = ('time',np.arange(0,len(ds.time)))

hover_cols = ['eps1_bat','eps1_kra','eps2_bat','eps2_kra', 'chi1']
arguments = {'hover_cols':hover_cols,'legend':'left','groupby':'itime','loglog':True}

obs = ds.hvplot(x='k_rpm',y='corrdTdzsp1_rpm',label='Observed',**arguments)
noi = ds.hvplot(x='k_rpm',y='noise_rpm',label='Noise',line_dash='dotted',**arguments)
bat = ds.hvplot(x='k_rpm',y='bat1',label='Batchelor',**arguments)
krai = ds.hvplot(x='k_rpm',y='kra1',label='Kraichnan',**arguments)
power = ds.hvplot(x='k_rpm',y='power1',label='Power law',line_dash='dashed',**arguments)
rc = ds.hvplot(x='k_rpm',y='bat1_rc',label='Ren-Chieh',**arguments)

In [ ]:
b = (obs*bat*krai*rc*power*noi).options(height=500,width=400,ylim=(1e-9, 1e-2),xlim=(1e1,2e3),
                      title=f'Constant DoF',
                      xlabel=r'k_z',ylabel='dPhidz',
                      fontsize={'title': 16, 'labels': 14, 'xticks': 12, 'yticks': 12, 'legend':12})

In [ ]:
a